In [451]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

# Plotly
import plotly.offline as py;
from plotly import tools;
import plotly.graph_objs as go;
import plotly.express as px
from plotly.subplots import make_subplots

import folium 

import warnings;
warnings.filterwarnings('ignore')
py.init_notebook_mode(connected=True)

Loading data for Paris and Lyon and creating a new dataframe with all the data so I can use all three of them depending on the analysis/graph that I want to show. 

In [360]:
paris = pd.read_csv(r'C:\Users\radek\Git-Hub\Final_Project_Ironhack\07 - EDA\Paris_for_EDA.csv')
lyon = pd.read_csv(r'C:\Users\radek\Git-Hub\Final_Project_Ironhack\07 - EDA\Lyon_for_EDA.csv')

In [361]:
paris.head(2)

,datetime,tempmax,tempmin,temp,humidity,precip,precipcover,windspeed,pressure,cloudcover,...,pm10,o3,no2,Clear,Partially cloudy,Rain,Snow,Overcast,month,day_of_week
0,2014-01-01,10.5,6.7,8.4,85.06,1.82,16.67,28.9,1006.1,26.7,...,30.0,22.0,42.0,0,1,1,0,0,1,2
1,2014-01-02,11.1,8.5,9.9,81.63,4.96,20.83,29.2,1001.7,19.7,...,33.0,22.0,34.0,0,0,1,0,0,1,3


In [362]:
lyon.head(2)

,datetime,tempmax,tempmin,temp,humidity,precip,precipcover,windspeed,pressure,cloudcover,...,pm10,o3,no2,Clear,Partially cloudy,Rain,Snow,Overcast,month,day_of_week
0,2014-01-01,11.9,8.2,9.8,68.37,0.33,8.33,42.0,1014.5,70.2,...,10.0,23.0,14.0,0,1,1,0,0,1,2
1,2014-01-02,12.1,8.4,10.5,78.29,11.94,37.50,35.5,1010.2,59.2,...,11.0,10.0,15.0,0,1,1,0,0,1,3


To clean a little the data I will remove some columns and add year and target column before joining the datframes

In [363]:
cols_to_drop = ['tempmax', 'tempmin', 'precipcover']
lyon.drop(columns=cols_to_drop, inplace=True)
paris.drop(columns=cols_to_drop, inplace=True)


paris.datetime= pd.to_datetime(paris.datetime)
lyon.datetime= pd.to_datetime(lyon.datetime)

paris['year']=paris.datetime.dt.year
lyon['year']=lyon.datetime.dt.year

lyon['target']=pd.cut(lyon.pm25, bins=[0,50,100,200], labels=[0,1,2])
paris['target']=pd.cut(paris.pm25, bins=[0,50,100,200], labels=[0,1,2])

In [364]:
paris.head(2)

,datetime,temp,humidity,precip,windspeed,pressure,cloudcover,visibility,uvindex,pm25,...,no2,Clear,Partially cloudy,Rain,Snow,Overcast,month,day_of_week,year,target
0,2014-01-01,8.4,85.06,1.82,28.9,1006.1,26.7,24.8,0.0,43.0,...,42.0,0,1,1,0,0,1,2,2014,0
1,2014-01-02,9.9,81.63,4.96,29.2,1001.7,19.7,24.3,0.0,53.0,...,34.0,0,0,1,0,0,1,3,2014,1


In [365]:
lyon.head(2)

,datetime,temp,humidity,precip,windspeed,pressure,cloudcover,visibility,uvindex,pm25,...,no2,Clear,Partially cloudy,Rain,Snow,Overcast,month,day_of_week,year,target
0,2014-01-01,9.8,68.37,0.33,42.0,1014.5,70.2,55.1,0.0,26.0,...,14.0,0,1,1,0,0,1,2,2014,0
1,2014-01-02,10.5,78.29,11.94,35.5,1010.2,59.2,46.5,0.0,31.0,...,15.0,0,1,1,0,0,1,3,2014,0


joining two dataframes together

In [366]:
paris['city']='Paris'
lyon['city']='Lyon'
data = pd.concat([paris,lyon])

In [367]:
data.shape

(5844, 23)

## Graphs on hystorical data

### Pie chart target by year 

In [368]:
years_target_paris= pd.crosstab(paris.target, paris.year)
years_target_paris=years_target_paris.reset_index(drop=True)
years_target_paris

year,2014,2015,2016,2017,2018,2019,2020,2021
0,32,41,72,71,88,146,204,185
1,291,280,261,263,254,201,154,172
2,42,44,33,31,23,18,8,8


In [369]:
years_target_lyon= pd.crosstab(lyon.target, lyon.year)
years_target_lyon=years_target_lyon.reset_index(drop=True)
years_target_lyon

year,2014,2015,2016,2017,2018,2019,2020,2021
0,214,152,217,217,261,268,258,258
1,136,193,128,136,101,84,96,103
2,15,20,21,12,3,13,12,4


In [370]:
year_pie=2020
# year_pie=str(year_pie)

In [371]:
fig_pie_paris = px.pie(years_target_paris, values=year_pie, names=['Good', 'Moderate', 'Bad'],color=['Good', 'Moderate', 'Bad'],color_discrete_map=	{'Good':'#24C0C0','Moderate':'#C0B724','Bad':'#C02D24',}, hole=.3,title=f'Quality of air in Paris in {year_pie}')

fig_pie_paris.show()

In [442]:
fig_pie = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
labels= ['Good','Moderate','Bad']

colors= [ 'rgb(79, 129, 102)', 
                 'rgb(36, 73, 147)','rgb(175, 49, 35)' ]
fig_pie.add_trace(go.Pie(labels=labels, values=years_target_paris[year_pie], name="Paris",  marker_colors=colors),
              1, 1)
fig_pie.add_trace(go.Pie(labels=labels, values=years_target_lyon[year_pie], name="Lyon", marker_colors=colors),
              1, 2)


# Use `hole` to create a donut-like pie chart
fig_pie.update_traces(hole=.35, hoverinfo="label+percent+name")

fig_pie.update_layout(
    title_text=f"Percentage of days with vairous Air quality in Paris and Lyon in {year_pie}",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='PARIS', x=0.19, y=0.5, font_size=20, showarrow=False),
                 dict(text='LYON', x=0.8, y=0.5, font_size=20, showarrow=False)])

fig_pie.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig_pie.show()

Year line chart

In [373]:
year_chart =year_pie
pollutant = 'pm25'

In [374]:
line_paris = paris[['datetime', pollutant]].loc[(paris.year==year_chart)]
line_lyon = lyon[['datetime', pollutant]].loc[(lyon.year==year_chart)]


In [375]:
fig_pollutant_year = go.Figure()
fig_pollutant_year.add_trace(go.Line(
    x=line_paris['datetime'],
    y=line_paris[pollutant],
    name='Paris',
    line_color='#2A6C6C'
))
fig_pollutant_year.add_trace(go.Line(
    x=line_lyon['datetime'],
    y=line_lyon[pollutant],
    name='Lyon',
    line_color='#B9DC36'
))

name_poll = {'pm25':'PM2.5', 'pm10':'PM10','o3':'ozone','no2': 'nitrogen dioxide'}
# Layout
fig_pollutant_year.update_layout(barmode='group', xaxis_tickangle=-45,title=f"Yearly concentration of {name_poll[pollutant]} in {year_chart}",title_x=0.5,xaxis_title="Date",
    yaxis_title="Concentartion",)
# Plot
fig_pollutant_year.show()

Monthly table + small graph

In [376]:
data.head()

,datetime,temp,humidity,precip,windspeed,pressure,cloudcover,visibility,uvindex,pm25,...,Clear,Partially cloudy,Rain,Snow,Overcast,month,day_of_week,year,target,city
0,2014-01-01,8.4,85.06,1.82,28.9,1006.1,26.7,24.8,0.0,43.0,...,0,1,1,0,0,1,2,2014,0,Paris
1,2014-01-02,9.9,81.63,4.96,29.2,1001.7,19.7,24.3,0.0,53.0,...,0,0,1,0,0,1,3,2014,1,Paris
2,2014-01-03,11.0,76.53,0.14,29.2,1006.0,20.1,20.4,0.0,50.0,...,0,1,1,0,0,1,4,2014,0,Paris
3,2014-01-04,9.1,80.53,0.67,21.5,1001.5,22.2,21.6,0.0,52.0,...,0,1,1,0,0,1,5,2014,1,Paris
4,2014-01-05,6.9,82.81,1.25,22.8,1007.5,22.8,24.0,0.0,45.0,...,0,1,1,0,0,1,6,2014,0,Paris


In [377]:
columns= ['temp', 'humidity', 'precip', 'windspeed', 'pressure', 'cloudcover','visibility', 'uvindex','year','month','pm25','pm10','o3','no2','city']

In [378]:
#Select month
month_chart =12

In [379]:
table = data[columns][(data[columns]['year']==year_chart) &(data[columns]['month']==month_chart)].groupby(['city']).agg('mean').round()

In [380]:
table

,temp,humidity,precip,windspeed,pressure,cloudcover,visibility,uvindex,year,month,pm25,pm10,o3,no2
city,,,,,,,,,,,,,,
Lyon,6.0,82.0,5.0,28.0,1011.0,73.0,30.0,2.0,2020.0,12.0,42.0,14.0,20.0,11.0
Paris,7.0,86.0,5.0,20.0,1007.0,22.0,17.0,1.0,2020.0,12.0,48.0,21.0,16.0,22.0


In [381]:
table_weather= table[['temp', 'humidity', 'precip', 'windspeed', 'pressure', 'cloudcover','visibility', 'uvindex']]
table_weather

,temp,humidity,precip,windspeed,pressure,cloudcover,visibility,uvindex
city,,,,,,,,
Lyon,6.0,82.0,5.0,28.0,1011.0,73.0,30.0,2.0
Paris,7.0,86.0,5.0,20.0,1007.0,22.0,17.0,1.0


In [446]:
table_weather.at['Paris','humidity']

86.0

In [382]:
table_pollutant = table[['pm25','pm10','o3','no2']]
table_pollutant=table_pollutant.T
table_pollutant

city,Lyon,Paris
pm25,42.0,48.0
pm10,14.0,21.0
o3,20.0,16.0
no2,11.0,22.0


In [383]:
fig_bar_month = go.Figure()

fig_bar_month.add_trace(
	go.Bar(x=['PM2.5','PM10','Ozone', 'Nitrogen dioxide'], y=table_pollutant['Lyon'], name='Lyon',marker_color='#B9DC36'),
)
fig_bar_month.add_trace(
    go.Bar(x=['PM2.5','PM10','Ozone', 'Nitrogen dioxide'], y=table_pollutant['Paris'], name='Paris',marker_color='#2A6C6C'),
)


month_dict={1:'January',2:'Ferbuary',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'}
name_poll = {'pm25':'PM2.5', 'pm10':'PM10','o3':'ozone','no2': 'nitrogen dioxide'}
# Layout
fig_bar_month.update_layout(barmode='group', xaxis_tickangle=-45,title=f"Concentration of analysed pollutants in {month_dict[month_chart]} {year_chart}",title_x=0.5,xaxis_title="Pollutant",
    yaxis_title="Concentartion",)
# Plot
fig_bar_month.show()

## Time Series analysis 

Loading data

In [384]:
lyon_2 = pd.read_csv(r'C:\Users\radek\Git-Hub\Final_Project_Ironhack\07 - EDA\sec2_Lyon.csv')
paris_2 =pd.read_csv(r'C:\Users\radek\Git-Hub\Final_Project_Ironhack\07 - EDA\sec2_Paris.csv')

In [385]:
lyon_2.head(2)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,...,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,y,poll,year,month,day_of_week
0,2014-01-01,48.389216,25.566714,80.493283,48.389216,48.389216,5.126439,5.126439,5.126439,0.556680,...,4.569759,0.0,0.0,0.0,53.515654,26.0,pm25,2014,1,Wednesday
1,2014-01-02,48.401781,27.984405,82.082121,48.401781,48.401781,6.611617,6.611617,6.611617,2.240415,...,4.371202,0.0,0.0,0.0,55.013398,31.0,pm25,2014,1,Thursday


In [386]:
paris_2.head(2)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,...,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,y,poll,year,month,day_of_week
0,2014-01-01,73.307396,52.855371,101.634306,73.307396,73.307396,3.804925,3.804925,3.804925,1.485682,...,2.319244,0.0,0.0,0.0,77.112322,43.0,pm25,2014,1,Wednesday
1,2014-01-02,73.310192,51.393793,101.805344,73.310192,73.310192,4.045865,4.045865,4.045865,1.237323,...,2.808543,0.0,0.0,0.0,77.356057,53.0,pm25,2014,1,Thursday


In [387]:
pollutant_2 = 'pm10'

In [388]:
fit_pol = lyon_2.loc[lyon_2.poll==pollutant_2][['ds','yhat_lower','yhat_upper','yhat','y','month','day_of_week']]

In [389]:
name_poll = {'pm25':'PM2.5', 'pm10':'PM10','o3':'ozone','no2': 'nitrogen dioxide'}

fig_lyon_model = go.Figure([
    go.Scatter(
        name=name_poll[pollutant_2],
        x=fit_pol['ds'],
        y=fit_pol['yhat'],
        mode='lines',
        line=dict(color='#B9DC36'),
    ),
    go.Scatter(
        name='Upper Bound',
        x=fit_pol['ds'],
        y=fit_pol['yhat_upper'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x=fit_pol['ds'],
        y=fit_pol['yhat_lower'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ),
    go.Scatter(
        name='Real data',
        x=fit_pol['ds'],
        y=fit_pol['y'],
        mode='markers',
        marker=dict(color="#FF5733",size=2),
        showlegend=False
)
])
fig_lyon_model.update_layout(
    yaxis_title='Concentration',
    title='Fitting of data to the forecast model - Lyon',
    hovermode="x"
)
fig_lyon_model.show()

In [390]:
fit_por = paris_2.loc[paris_2.poll==pollutant_2][['ds','yhat_lower','yhat_upper','yhat','y']]

In [449]:
name_poll = {'pm25':'PM2.5', 'pm10':'PM10','o3':'ozone','no2': 'nitrogen dioxide'}

fig_paris_model = go.Figure([
    go.Scatter(
        name=name_poll[pollutant],
        x=fit_por['ds'],
        y=fit_por['yhat'],
        mode='lines',
        line=dict(color='#2A6C6C'),
    ),
    go.Scatter(
        name='Upper Bound',
        x=fit_por['ds'],
        y=fit_por['yhat_upper'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x=fit_por['ds'],
        y=fit_por['yhat_lower'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(97, 169, 136, 0.3)',
        fill='tonexty',
        showlegend=False
    ),
    go.Scatter(
        name='Real data',
        x=fit_por['ds'],
        y=fit_por['y'],
        mode='markers',
        marker=dict(color="#FF5733",size=2),
        showlegend=False
)
])
fig_paris_model.update_layout(
    yaxis_title='Concentration',
    title=f'Fitting of {name_poll[pollutant]} data to the forecast model - Paris',
    hovermode="x"
)
fig_paris_model.show()

In [392]:
fit_2022l=lyon_2.loc[(lyon_2.poll==pollutant_2) & (lyon_2.year==2022)][['ds','yhat_lower','yhat_upper','yhat','y']]

In [393]:
fig_lyon_forecast = go.Figure([
    go.Scatter(
        name=name_poll[pollutant_2],
        x=fit_2022l['ds'],
        y=fit_2022l['yhat'],
        mode='lines',
        line=dict(color='#B9DC36'),
    ),
    go.Scatter(
        name='Upper Bound',
        x=fit_2022l['ds'],
        y=fit_2022l['yhat_upper'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x=fit_2022l['ds'],
        y=fit_2022l['yhat_lower'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ),
    ])
fig_lyon_forecast.update_layout(
    yaxis_title='Concentration',
    title=f'Forecatast of {name_poll[pollutant_2]} pollution in Lyon in 2022',
    hovermode="x"
)
fig_lyon_forecast.show()

In [394]:
fit_2022p=paris_2.loc[(paris_2.poll==pollutant_2) & (paris_2.year==2022)][['ds','yhat_lower','yhat_upper','yhat','y']]

In [441]:
fig_paris_forecast = go.Figure([
    go.Scatter(
        name=name_poll[pollutant_2],
        x=fit_2022p['ds'],
        y=fit_2022p['yhat'],
        mode='lines',
        line=dict(color='#2A6C6C'),
    ),
    go.Scatter(
        name='Upper Bound',
        x=fit_2022p['ds'],
        y=fit_2022p['yhat_upper'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x=fit_2022p['ds'],
        y=fit_2022p['yhat_lower'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(60, 175, 20, 0.3)',
        fill='tonexty',
        showlegend=False
    ),
    ])
fig_paris_forecast.update_layout(
    yaxis_title='Concentration',
    title=f'Forecatast of {name_poll[pollutant_2]} pollution in Paris in 2022',
    hovermode="x"
)
fig_paris_forecast.show()

In [396]:
lyon_2.head(2)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,...,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,y,poll,year,month,day_of_week
0,2014-01-01,48.389216,25.566714,80.493283,48.389216,48.389216,5.126439,5.126439,5.126439,0.556680,...,4.569759,0.0,0.0,0.0,53.515654,26.0,pm25,2014,1,Wednesday
1,2014-01-02,48.401781,27.984405,82.082121,48.401781,48.401781,6.611617,6.611617,6.611617,2.240415,...,4.371202,0.0,0.0,0.0,55.013398,31.0,pm25,2014,1,Thursday


In [397]:
fit_tredl = lyon_2.loc[lyon_2.poll==pollutant_2][['ds','trend_lower','trend_upper','trend']]
fit_tredp = paris_2.loc[paris_2.poll==pollutant_2][['ds','trend_lower','trend_upper','trend']]

In [398]:
fig_trend = go.Figure([

#Lyon
    go.Scatter(
        name='Lyon',
        x=fit_tredl['ds'],
        y=fit_tredl['trend'],
        mode='lines',
        line=dict(color='#B9DC36'),
    ),
    go.Scatter(
        name='Upper Bound',
        x=fit_tredl['ds'][2920:],
        y=fit_tredl['trend_upper'][2920:],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x=fit_tredl['ds'][2920:],
        y=fit_tredl['trend_lower'][2920:],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ),

# Paris
	    go.Scatter(
        name='Paris',
        x=fit_tredp['ds'],
        y=fit_tredp['trend'],
        mode='lines',
        line=dict(color='#2A6C6C'),
    ),
    go.Scatter(
        name='Upper Bound',
        x=fit_tredp['ds'][2920:],
        y=fit_tredp['trend_upper'][2920:],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x=fit_tredp['ds'][2920:],
        y=fit_tredp['trend_lower'][2920:],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ),
    ])

fig_trend.update_layout(
    yaxis_title='Concentration',
    title=f'Trend of {name_poll[pollutant_2]} pollution in 2022',
    hovermode="x"
)
fig_trend.show()

In [403]:
fit_yearlyl = lyon_2.loc[(lyon_2.poll==pollutant_2)&(lyon_2.year==2016)][['ds','yearly','month']]
fit_yearlyp = paris_2.loc[(paris_2.poll==pollutant_2)&(paris_2.year==2016)][['ds','yearly','month']]

In [414]:
fit_yearlyl.ds=pd.to_datetime(fit_yearlyl.ds)
fit_yearlyp.ds=pd.to_datetime(fit_yearlyp.ds)

In [426]:
fit_yearlyl['month_day'] = fit_yearlyl['ds'].dt.dayofyear
fit_yearlyp['month_day'] = fit_yearlyp['ds'].dt.dayofyear

In [424]:
fit_yearlyp.dtypes

ds           datetime64[ns]
yearly              float64
month                 int64
month_day            object
dtype: object

In [428]:
averge_for_daysl = fit_yearlyl.groupby('month_day').agg('mean')
averge_for_daysp = fit_yearlyp.groupby('month_day').agg('mean')

In [429]:
averge_for_daysp.dtypes

yearly    float64
month     float64
dtype: object

In [431]:
fig_season_yearly = go.Figure()
fig_season_yearly.add_trace(go.Line(
    x=averge_for_daysp.index,
    y=averge_for_daysp['yearly'],
    name='Paris',
    line_color='#2A6C6C'
))
fig_season_yearly.add_trace(go.Line(
    x=averge_for_daysl.index,
    y=averge_for_daysl['yearly'],
    name='Lyon',
    line_color='#B9DC36'
))

name_poll = {'pm25':'PM2.5', 'pm10':'PM10','o3':'ozone','no2': 'nitrogen dioxide'}
# Layout
fig_season_yearly.update_layout(barmode='group', xaxis_tickangle=-45,title=f"Yearly seasonality of {name_poll[pollutant_2]}",title_x=0.5,xaxis_title="Day of year",
    yaxis_title="Yearly",)
# Plot
fig_season_yearly.show()

In [432]:
fit_weeklyl = lyon_2.loc[(lyon_2.poll==pollutant_2)&(lyon_2.year==2016)][['ds','weekly','day_of_week']]
fit_weeklyp = paris_2.loc[(paris_2.poll==pollutant_2)&(paris_2.year==2016)][['ds','weekly','day_of_week']]

In [433]:
fit_weeklyp.head()

,ds,weekly,day_of_week
730,2016-01-01,-1.009571,Friday
731,2016-01-02,-3.523874,Saturday
732,2016-01-03,-1.061726,Sunday
733,2016-01-04,0.372397,Monday
734,2016-01-05,1.717034,Tuesday


In [439]:
averge_daysl = fit_weeklyl.groupby('day_of_week').agg('mean')
averge_daysp = fit_weeklyp.groupby('day_of_week').agg('mean')
averge_daysp = averge_daysp.reindex(index = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
averge_daysl = averge_daysl.reindex(index = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
averge_daysp

,weekly
day_of_week,
Monday,0.372397
Tuesday,1.717034
Wednesday,1.383248
Thursday,2.122492
Friday,-1.009571
Saturday,-3.523874
Sunday,-1.061726


In [450]:
fig_season_weekly = go.Figure()
fig_season_weekly.add_trace(go.Line(
    x=averge_daysp.index,
    y=averge_daysp['weekly'],
    name='Paris',
    line_color='#2A6C6C'
))
fig_season_weekly.add_trace(go.Line(
    x=averge_daysl.index,
    y=averge_daysl['weekly'],
    name='Lyon',
    line_color='#B9DC36'
))

name_poll = {'pm25':'PM2.5', 'pm10':'PM10','o3':'ozone','no2': 'nitrogen dioxide'}
# Layout
fig_season_weekly.update_layout(barmode='group', xaxis_tickangle=-45,title=f"Weekly seasonality of {name_poll[pollutant_2]}",title_x=0.5,xaxis_title="Day of year",
    yaxis_title="Weekly",)
# Plot
fig_season_weekly.show()

### Map

In [461]:
m= folium.Map(location=[46.2276, 2.2137], zoom_start=6,tiles = "Stamen Toner")

#create markers
folium.CircleMarker(
    location=[48.8566, 2.3522],
    radius = 4,
    popup="Paris",
	color='#2A6C6C',
    icon=folium.Icon(icon="cloud"),
).add_to(m)

folium.CircleMarker(
    location=[45.7640, 4.8357],
    popup="Lyon",
    radius = 4,
    icon=folium.Icon(color="green"),
).add_to(m)

m